In [21]:
# Import necessary libraries
import pandas as pd

In [22]:
# Load the data
businesses = pd.read_csv("data/businesses.csv")
new_businesses = pd.read_csv("data/new_businesses.csv")
countries = pd.read_csv("data/countries.csv")
categories = pd.read_csv("data/categories.csv")

In [23]:
# What is the oldest business on every continent?

# Start by merging the businesses and countries datasets into one
businesses_countries = businesses.merge(countries, on="country_code")

# Create a new DataFrame that lists only the continent and oldest year_founded
continent = businesses_countries.groupby("continent").agg({"year_founded":"min"})

# Merge this continent DataFrame with businesses_countries
merged_continent = continent.merge(businesses_countries, on=["continent", "year_founded"])

# Subset the continent DataFrame so that only the four columns of interest are included, saving it as oldest_business_continent
oldest_business_continent = merged_continent[["continent", "country", "business", "year_founded"]]

# View the result
print(oldest_business_continent)

       continent    country                     business  year_founded
0         Africa  Mauritius               Mauritius Post          1772
1           Asia      Japan                   Kongō Gumi           578
2         Europe    Austria  St. Peter Stifts Kulinarium           803
3  North America     Mexico  La Casa de Moneda de México          1534
4        Oceania  Australia               Australia Post          1809
5  South America       Peru      Casa Nacional de Moneda          1565


In [24]:
# How many countries per continent lack data on the oldest businesses? 
# Does including the `new_businesses` data change this?

# Add the data in new_businesses to the existing businesses
all_businesses = pd.concat([new_businesses, businesses])

# Perform a new merge between the businesses and the countries data. Use additional parameters this time to perform an outer merge and create an indicator column to better see the missing values. An outer merge combines two DataFrames based on a key column and includes all rows from both DataFrames
new_all_countries = all_businesses.merge(countries, on="country_code", how="outer",  indicator=True)

# Filter to find countries with missing business data
new_missing_countries = new_all_countries[new_all_countries["_merge"] != "both"]

# Group by continent and create a "count_missing" column
count_missing = new_missing_countries.groupby("continent").agg({"country":"count"})
count_missing.columns = ["count_missing"]

# View the results
print(count_missing)

               count_missing
continent                   
Africa                     3
Asia                       7
Europe                     2
North America              5
Oceania                   10
South America              3


In [26]:
# Which business categories are best suited to last over the course of centuries?

# Start by merging the businesses and categories data into one DataFrame
businesses_categories = businesses.merge(categories, on="category_code")

# Merge all businesses, countries, and categories together
businesses_categories_countries = businesses_categories.merge(countries, on="country_code")

# Create the oldest by continent and category DataFrame
oldest_by_continent_category = businesses_categories_countries.groupby(["continent", "category"]).agg({"year_founded":"min"})
oldest_by_continent_category

year_founded
continent     category                                       
Africa        Agriculture                                1947
              Aviation & Transport                       1854
              Banking & Finance                          1892
              Distillers, Vintners, & Breweries          1933
              Energy                                     1968
              Food & Beverages                           1878
              Manufacturing & Production                 1820
              Media                                      1943
              Mining                                     1962
              Postal Service                             1772
Asia          Agriculture                                1930
              Aviation & Transport                       1858
              Banking & Finance                          1830
              Cafés, Restaurants & Bars                  1153
              Conglomerate                               1841
              Construction                                578
              Defense                                    1808
              Distillers, Vintners, & Breweries          1853
              Energy                                     1928
              Food & Beverages                           1820
              Manufacturing & Production                 1736
              Media                                      1931
              Mining                                     1913
              Postal Service                             1800
              Retail                                     1883
              Telecommunications                         1885
              Tourism & Hotels                           1584
Europe        Agriculture                                1218
              Banking & Finance                          1606
              Cafés, Restaurants & Bars                   803
              Consumer Goods                             1649
              Defense                                    1878
              Distillers, Vintners, & Breweries           862
              Manufacturing & Production                  864
              Media                                      1999
              Medical                                    1422
              Mining                                     1248
              Postal Service                             1520
              Telecommunications                         1912
              Tourism & Hotels                           1230
North America Agriculture                                1638
              Aviation & Transport                       1870
              Banking & Finance                          1891
              Distillers, Vintners, & Breweries          1703
              Food & Beverages                           1920
              Manufacturing & Production                 1534
              Media                                      1909
              Retail                                     1670
              Tourism & Hotels                           1770
Oceania       Banking & Finance                          1861
              Postal Service                             1809
South America Banking & Finance                          1565
              Cafés, Restaurants & Bars                  1877
              Defense                                    1811
              Food & Beverages                           1660
              Manufacturing & Production                 1621

In [27]:
businesses_categories_countries["continent"].value_counts()

continent
Africa           50
Asia             42
Europe           42
North America    17
South America     9
Oceania           3
Name: count, dtype: int64